<a href="https://colab.research.google.com/github/LeandraOS/Sockets/blob/master/C%C3%B3pia_de_Copy_of_Introdu%C3%A7%C3%A3o_ao_agrupamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import altair as alt

# Agrupamento: conceito e exemplos

Um dos padrões importantes que percebemos e usamos para entender/descrever dados é o de *subconjuntoso dos dados com alta semelhança* entre si. No jargão que usaremos aqui, **grupos de dados** semelhantes entre si.

Algoritmos de agrupamento servem principalmente para ajudar um analista a reduzir um conjunto de dados a um conjunto menor de grupos. Isso facilita compreensão e comunicação.

Para exemplificar, primeiro usaremos dados do Rotten Tomatoes sobre avaliação da crítica e bilheteria dos filmes de Scarlett Johansson. Os dados foram raspados [daqui](https://www.rottentomatoes.com/celebrity/scarlett_johansson). Se você quiser, você pode raspar para outros atores/diretores com o código [desse repo](https://github.com/cienciadedados-ufcg/agrupamento-filmes).


In [11]:
filmes = pd.read_csv("https://raw.githubusercontent.com/cienciadedados-ufcg/agrupamento-filmes/master/data/movies.csv")
filmes.head()

,RATING,TITLE,CREDIT,BOXOFFICE,YEAR
0,85,Avengers: Infinity War,Natasha Romanoff/Black Widow,665.0,2018
1,44,Rough Night,Jess,22.1,2017
2,43,Ghost in the Shell,Major,40.6,2017
3,72,Sing,Ash,270.4,2016
4,91,Captain America: Civil War,Natasha Romanoff/Black Widow,408.1,2016


## Estrutura de grupos em 1d

In [12]:
alt.Chart(filmes).mark_point(
    size = 70
).encode(
    x = 'RATING',
    tooltip = ['TITLE', 'RATING']
).properties(
    width=600,
    height=250
).interactive()

alt.Chart(...)

In [13]:
alt.Chart(filmes).mark_point(
    size = 70
).encode(
    x = alt.X('BOXOFFICE', scale=alt.Scale(type='log')),
    tooltip = ['TITLE', 'RATING', 'BOXOFFICE']
).properties(
    width=600,
    height=150
).interactive()

alt.Chart(...)

## Estrutura em 2d

In [14]:
alt.Chart(filmes).mark_circle(
    size = 70
).encode(
    y = alt.Y('BOXOFFICE', scale=alt.Scale(type='log')),
    x = 'RATING',
    tooltip = ['TITLE', 'RATING', 'BOXOFFICE']
).interactive()

alt.Chart(...)

Aliás, se você quiser já ver a análise dos grupos nesses filmes feita por um jornalista, [basta ir aqui](https://fivethirtyeight.com/features/the-four-types-of-scarlett-johansson-movies/). 

## Encontrando agrupamentos

Uma forma de descrever estrutura nos dados é percebendo grupos de observações mais semelhantes entre si que com o restante dos dados.

O tipo de pergunta que queremos responder com agrupamento é _existem grupos de dados claramente diferentes em termos de var_1, var_2... e var_n?_

### Sempre há grupos?

Não. Para ilustrar, podemos gerar dados sintéticos com e sem tendência de grupos e compará-los 

In [15]:
from sklearn.datasets.samples_generator import make_blobs

def cria_df_exemplo(ngrupos, random_state=None): 
  X, y = make_blobs(n_samples=300, centers=ngrupos,
                    cluster_std=0.60, random_state = random_state)
  return pd.DataFrame(dict(x=X[:,0], y=X[:,1]))
 
df1 = cria_df_exemplo(4)
df1.head()

,x,y
0,-0.325667,-4.470159
1,-8.441421,8.381118
2,-6.484716,-5.509827
3,-7.595721,9.442472
4,8.140929,3.659936


Rode várias vezes as células seguintes. A primeira quase sempre mostrará uma estrutura com 4 grupos. A segunda deve dificultar vermos grupos.

In [18]:
alt.Chart(cria_df_exemplo(4)).mark_point(
    size = 70
).encode(
    x = 'x',
    y = 'y'
).interactive()

alt.Chart(...)

In [19]:
alt.Chart(df2).mark_point(
    size = 70
).encode(
    x = 'x',
    y = 'y'
).interactive()

NameError: ignored



---

## Usando um algoritmo para encontrar grupos

Há vários algoritmos para uma mesma definição de objetivo. Começaremos com uma situação onde o número de grupos a encontrar é claro e cada ponto pertencerá a um grupo.

Usaremos 2 exemplos: o sintético e o dos filmes.

Primeiro o sintético:


In [ ]:
df_ex = cria_df_exemplo(4, random_state=0)
alt.Chart(df_ex).mark_circle(
    size = 70
).encode(
    x = 'x',
    y = 'y'
).interactive()

Encontrando os grupos

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(df_ex)
grupo = kmeans.predict(df_ex)

df_agrupado = df_ex.assign(grupo = grupo)

df_agrupado

Visualizando os grupos na cor dos pontos

In [ ]:
alt.Chart(df_agrupado).mark_circle(
    size = 70
).encode(
    x = 'x',
    y = 'y', 
    color = 'grupo:N'
).interactive()

### Agora os filmes

Como as variáveis têm escalas muito diferentes, precisaremos pré-processar os dados normalizando as variáveis.


In [ ]:
alt.Chart(filmes).mark_circle(
    size = 70
).encode(
    y = alt.Y('BOXOFFICE', scale=alt.Scale(type='log')),
    x = 'RATING',
    tooltip = ['TITLE', 'RATING', 'BOXOFFICE']
).interactive()

In [ ]:
from sklearn import preprocessing

filmes_t = pd.DataFrame({'rating' : filmes['RATING'], 
                         'boxoffice' : np.log10(filmes['BOXOFFICE'])})

ss = preprocessing.StandardScaler()
filmes_scaled = pd.DataFrame(ss.fit_transform(filmes_t),columns = filmes_t.columns)
print(filmes_scaled.head(5))

alt.Chart(filmes_scaled).mark_circle(
    size = 100, 
    stroke = 'black'
).encode(
    y = 'boxoffice',
    x = 'rating'
)

Repare nas escalas dos eixos acima.

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(filmes_scaled)
grupo = kmeans.predict(filmes_scaled)

filmes_scaled_ag = filmes_scaled.assign(grupo = grupo, TITLE = filmes['TITLE'])
filmes_agrupado = filmes.assign(grupo = grupo)

print(filmes_scaled_ag.head())
filmes_agrupado.head()

In [ ]:
alt.Chart(filmes_scaled_ag).mark_circle(
    size = 100, 
    stroke = 'black'
).encode(
    y = 'boxoffice',
    x = 'rating',
    color = 'grupo:N',
    tooltip = ['TITLE', 'rating', 'boxoffice']
).interactive()

In [ ]:
alt.Chart(filmes_agrupado).mark_circle(
    size = 100, 
    stroke = 'black'
).encode(
    y = alt.Y('BOXOFFICE', scale=alt.Scale(type='log')),
    x = 'RATING',
    color = 'grupo:N',
    tooltip = ['TITLE', 'RATING', 'BOXOFFICE']
).interactive()


---

## Variando o número de grupos a encontrar



In [ ]:
#@title Grupos a encontrar: k
k = 9 #@param {type:"slider", min:1, max:10, step:1}

kmeans = KMeans(n_clusters=k, n_init=50)
kmeans.fit(filmes_scaled)
grupo = kmeans.predict(filmes_scaled)

filmes_scaled_ag = filmes_scaled.assign(grupo = grupo, TITLE = filmes['TITLE'])

alt.Chart(filmes_scaled_ag).mark_circle(
    size = 120, 
    opacity = 1
).encode(
    y = 'boxoffice',
    x = 'rating',
    color = 'grupo:N',
    tooltip = ['TITLE', 'rating', 'boxoffice']
).interactive()

In [ ]:
#@title Grupos a encontrar: k
k = 9 #@param {type:"slider", min:1, max:10, step:1}

kmeans = KMeans(n_clusters=4)

kmeans = KMeans(n_clusters=k, n_init=50)
kmeans.fit(df_ex)
grupo = kmeans.predict(df_ex)

df_agrupado = df_ex.assign(grupo = grupo)

alt.Chart(df_agrupado).mark_circle(
    size = 120, 
    opacity = .8
).encode(
    y = 'y',
    x = 'x',
    color = 'grupo:N'
)



---

## Leia mais

### Exemplos de agrupamentos

Nossa análise de filmes foi inspirada [nas análises baseadas em agrupamento de filmes do FiveThirtyEight](https://fivethirtyeight.com/tag/hollywood-taxonomy/). Leia lá algumas pra ver como eles as usam. 

### Algoritmos de agrupamento 

Usamos um algoritmo popular aqui. Existem **muitos** outros. Eles variam em várias dimensões: cada ponto só pertence a um grupo no final? O analista precisa decidir o número de grupos? Existem pontos que não pertencem a nenhum grupo? A heurística funciona como?

Normalmente, para **produzir** uma solução de agrupamento precisamos de: 

* Definição de proximidade/distância entre pontos
* Definição de proximidade/distância entre grupos ou grupos e pontos
* Processo de agrupamento 
* Decidir quantos grupos existem

Depois vem o principal: **avaliar e interpretar** a solução. _Agrupamento sempre dá um resultado, mas nem sempre o resultado é útil_. 

Há duas maneiras principais de agrupar: aglomerativa ou baseada em partição. 

O algoritmo que usamos para agrupar é um algoritmo que: dado um número de grupos e uma função de distância entre os pontos, procura uma partição dos dados na qual os grupos são homogêneos dentro de si e diferentes entre si. O nome desse algoritmo é K-means.

## Sobre o K-means

O algoritmo é conceitualmente bem simples; é uma iteração com duas fases: dado um conjunto de *k* centros de grupos (i) atribui cada ponto ao grupo cujo centro está mais próximo e (ii) move os centros para o meio dos pontos que compõem seu grupo. 

Há [uma boa visualização de como o algoritmo funciona aqui](http://tech.nitoyon.com/en/blog/2013/11/07/k-means/). E uma mais interativa [nesse outro link](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/).

